<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-01 13:24:43
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.37 C
-------------------
Today PnL: -41.70 K (-0.3%)
Current PnL: -18.37 L (-12.49%)
CY Booked + Current PnL: -10.10 L (-6.87%)
-------------------
Total profit:  2.51 L
Total loss:  -20.88 L
-------------------
Total Booked + Current PnL: 16.88 L (14.06%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.06%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 76.69 L (56.18%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-21.55,H-LC,5.24,108160.0,3848.0,13693.0,-1.03,3.69,12.66,16.81,10.0,0.28,0.81,20.33,X5K,ATH,METALS
71,SIS,477.00,477.00,2346.11,H-SC,6.83,65807.0,-15283.0,15280.0,0.18,-18.85,23.22,0.00,157.0,-1.00,0.49,33.37,OX40N,NTT,MISC
27,DIXON,14367.00,18940.15,-26.26,H-MC,6.13,135744.0,20808.0,15773.0,0.75,18.10,11.62,31.83,85.0,1.32,1.02,52.77,X40N,ATH,IT
55,MEDANTA,1087.93,1486.00,21.96,H-SC,12.59,156240.0,30040.0,16140.0,2.72,23.80,10.33,36.59,156.0,1.86,1.17,39.30,XY24,NTT,HEALTHCARE
67,SBIN,760.30,863.00,-12.46,M-LC,4.17,211298.0,9818.0,17390.0,0.10,4.87,8.23,13.51,62.0,0.56,1.59,18.08,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,2347.98,4546.37,7.43,H-SC,19.87,99975.0,-988.0,95516.0,-0.99,-0.98,95.54,93.63,149.0,-0.01,0.75,48.91,SR,ATH,CHEMICALS
41,ICICIPRULI,600.83,790.00,-15.89,H-MC,1.33,135826.0,38.0,42717.0,-2.43,0.03,31.45,31.48,111.0,0.00,1.02,12.15,X40,ATH,INSURANCE
35,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.67,150944.0,31.0,57268.0,1.22,0.02,37.94,37.97,90.0,0.00,1.14,25.33,AR,ATH,AUTO
91,WIPRO,243.46,420.00,-11.94,M-LC,5.19,150896.0,-49.0,109505.0,-1.98,-0.03,72.57,72.51,55.0,-0.00,1.13,5.99,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-21.82,H-LC,1.70,205582.0,1082.0,40335.0,-3.47,0.53,19.62,20.25,9.0,0.03,1.55,9.31,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-5.37,H-LC,2.82,182214.0,-18096.0,78079.0,-0.87,-9.03,42.85,29.94,24.0,-0.23,1.37,25.88,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.62,H-LC,2.87,185550.0,-14956.0,63736.0,-0.64,-7.46,34.35,24.33,30.0,-0.23,1.40,7.00,X40N,BTT,FINANCE
5,ASIANPAINT,2961.56,4250.0,-16.37,H-LC,3.15,166192.0,-35194.0,122816.0,2.00,-17.48,73.90,43.51,19.0,-0.29,1.25,15.69,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-15.89,H-LC,4.51,266772.0,1346.0,55569.0,1.44,0.51,20.83,21.44,26.0,0.02,2.01,7.18,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.90,H-SC,12.90,133006.0,7709.0,51367.0,2.94,6.15,38.62,47.15,144.0,0.15,1.00,66.40,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,21.96,H-SC,12.59,156240.0,30040.0,16140.0,2.72,23.80,10.33,36.59,156.0,1.86,1.17,39.30,XY24,NTT,HEALTHCARE
26,DEN,52.79,75.00,115.03,M-SC,3.29,60736.0,-23728.0,59266.0,2.57,-28.09,97.58,42.07,238.0,-0.40,0.46,28.98,AR,NTT,ENTERTAINMENT
17,BSOFT,628.85,836.99,-16.31,H-SC,12.67,70228.0,-39821.0,76247.0,2.50,-36.18,108.57,33.10,129.0,-0.52,0.53,16.69,XR,ATH,IT
68,SFL,1016.35,1287.00,32.15,M-SC,13.20,174421.0,-87797.0,157624.0,2.08,-33.48,90.37,26.63,231.0,-0.56,1.31,8.45,XY24,NTT,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-307.11,H-SC,20.41,88140.0,-45146.0,129742.0,-4.14,-33.87,147.20,63.47,147.0,-0.35,0.66,47.74,XR,NTT,CHEMICALS
21,CIPLA,1492.70,1795.00,-21.82,H-LC,1.70,205582.0,1082.0,40335.0,-3.47,0.53,19.62,20.25,9.0,0.03,1.55,9.31,X40N,BTT,PHARMA
60,RAJESHEXPO,517.65,518.00,1680.30,L-SC,3.64,48569.0,-88608.0,88702.0,-2.75,-64.59,182.63,0.07,266.0,-1.00,0.37,20.79,OX40N,NTT,JEWELLERY
40,ICICIGI,1839.64,2260.25,-15.24,H-LC,5.06,138787.0,2654.0,28465.0,-2.67,1.95,20.51,22.86,21.0,0.09,1.04,14.81,X40,ATH,INSURANCE
41,ICICIPRULI,600.83,790.00,-15.89,H-MC,1.33,135826.0,38.0,42717.0,-2.43,0.03,31.45,31.48,111.0,0.00,1.02,12.15,X40,ATH,INSURANCE


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,5.79,149271.0,-25288.0,25316.0,0.32,-14.49,16.96,0.02,135.0,-1.00,1.12,29.44,OX40N,NTT,PAINTS
75,SYMPHONY,1306.42,1306.0,-32.79,M-SC,8.90,143144.0,-27997.0,27942.0,-1.99,-16.36,19.52,-0.03,197.0,-1.00,1.08,3.59,OX40N,NTT,DURABLES
77,TATAELXSI,7332.28,7332.0,-18.99,H-MC,1.91,85113.0,-17539.0,17533.0,-0.23,-17.09,20.60,-0.00,101.0,-1.00,0.64,29.38,OX40N,NTT,IT
49,KANSAINER,299.63,340.0,-68.02,H-SC,1.70,223092.0,-46575.0,82901.0,1.22,-17.27,37.16,13.47,136.0,-0.56,1.68,12.84,XY24,NTT,PAINTS
83,UJJIVANSFB,52.77,53.0,35.36,H-SC,20.81,117612.0,-24867.0,25487.0,-1.60,-17.45,21.67,0.44,163.0,-0.98,0.88,38.81,OX40N,NTT,BANKS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-28.02,H-LC,13.87,234733.0,-61201.0,155112.0,-0.90,-20.68,66.08,31.73,1.0,-0.39,1.77,0.00,X40,BTT,IT
88,VBL,492.64,671.64,-8.67,H-LC,11.74,277231.0,12191.0,84112.0,-1.30,4.60,30.34,36.33,3.0,0.14,2.08,18.51,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-17.80,H-LC,7.46,263774.0,2173.0,143440.0,-2.35,0.83,54.38,55.67,4.0,0.02,1.98,6.99,X40,BTT,IT
1,ABB,5551.76,7934.00,-40.55,H-LC,7.62,212160.0,-4359.0,97275.0,-1.27,-2.01,45.85,42.91,5.0,-0.04,1.60,10.90,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.09,H-LC,7.46,177461.0,-13185.0,59698.0,-0.91,-6.92,33.64,24.40,8.0,-0.22,1.33,7.92,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,21.96,H-SC,12.59,156240.0,30040.0,16140.0,2.72,23.8,10.33,36.59,156.0,1.86,1.17,39.3,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.90,H-SC,12.90,133006.0,7709.0,51367.0,2.94,6.15,38.62,47.15,144.0,0.15,1.00,66.40,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,21.96,H-SC,12.59,156240.0,30040.0,16140.0,2.72,23.80,10.33,36.59,156.0,1.86,1.17,39.30,XY24,NTT,HEALTHCARE
25,DABUR,505.20,735.00,-2.87,H-MC,5.17,210308.0,13280.0,76342.0,1.94,6.74,36.30,45.49,100.0,0.17,1.58,20.66,XY24,BTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.67,150944.0,31.0,57268.0,1.22,0.02,37.94,37.97,90.0,0.00,1.14,25.33,AR,ATH,AUTO
19,CAMS,3643.00,5226.82,-8.14,H-SC,0.80,103160.0,1156.0,43193.0,-1.41,1.13,41.87,43.48,120.0,0.03,0.78,18.10,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.90,H-SC,12.90,133006.0,7709.0,51367.0,2.94,6.15,38.62,47.15,144.0,0.15,1.00,66.40,MH,ATH,POWER
90,WHIRLPOOL,1219.98,2270.00,-40.32,M-SC,7.03,100388.0,8890.0,69860.0,0.29,9.72,69.59,86.07,211.0,0.13,0.75,45.22,XR,NTT,DURABLES
42,INDIAMART,2327.09,4850.92,-46.96,H-SC,13.38,137842.0,14506.0,119261.0,0.45,11.76,86.52,108.45,118.0,0.12,1.04,36.74,AR,ATH,MISC
91,WIPRO,243.46,420.00,-11.94,M-LC,5.19,150896.0,-49.0,109505.0,-1.98,-0.03,72.57,72.51,55.0,-0.00,1.13,5.99,XR,NTT,IT
35,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.67,150944.0,31.0,57268.0,1.22,0.02,37.94,37.97,90.0,0.00,1.14,25.33,AR,ATH,AUTO


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2346.11,H-SC,6.83,65807.0,-15283.0,15280.0,0.18,-18.85,23.22,0.00,157.0,-1.00,0.49,33.37,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.18,68774.0,-44775.0,84826.0,0.20,-39.43,123.34,35.27,267.0,-0.53,0.52,68.33,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-20.05,H-SC,16.68,90690.0,-7290.0,29157.0,-1.87,-7.44,32.15,22.32,151.0,-0.25,0.68,31.53,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-4.53,H-LC,8.57,95310.0,-2435.0,39830.0,-0.15,-2.49,41.79,38.26,40.0,-0.06,0.72,27.26,X40,BTT,FINANCE
3,ALKYLAMINE,2347.98,4546.37,7.43,H-SC,19.87,99975.0,-988.0,95516.0,-0.99,-0.98,95.54,93.63,149.0,-0.01,0.75,48.91,SR,ATH,CHEMICALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,21.96,H-SC,12.59,156240.0,30040.0,16140.0,2.72,23.80,10.33,36.59,156.0,1.86,1.17,39.30,XY24,NTT,HEALTHCARE
27,DIXON,14367.00,18940.15,-26.26,H-MC,6.13,135744.0,20808.0,15773.0,0.75,18.10,11.62,31.83,85.0,1.32,1.02,52.77,X40N,ATH,IT
51,LALPATHLAB,2838.53,3545.00,-23.96,H-SC,5.54,216199.0,23179.0,24863.0,0.90,12.01,11.50,24.89,128.0,0.93,1.63,37.90,X40N,NTT,HEALTHCARE
88,VBL,492.64,671.64,-8.67,H-LC,11.74,277231.0,12191.0,84112.0,-1.30,4.60,30.34,36.33,3.0,0.14,2.08,18.51,X40N,ATH,FMCG
69,SHALBY,261.39,327.00,970.18,M-SC,8.98,145592.0,-35551.0,81022.0,0.21,-19.63,55.65,25.10,233.0,-0.44,1.09,16.68,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.69
1,25,41.66
2,50,71.28


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.82
LC    37.06
MC    22.10
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.08
X40N     15.15
X40      13.15
XY25     11.76
XR        8.94
OX40N     7.76
AR        7.67
X5K       2.18
MH        1.73
X200      1.37
SR        1.19
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.04
H-SC    27.28
H-MC    19.16
M-SC    12.09
M-LC     6.98
M-MC     2.60
L-SC     1.45
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.44,-0.41,32.83
IT,11.92,-17.63,74.77
FINANCE,8.35,-17.41,60.02
BANKS,7.39,-15.84,63.10
PAINTS,5.79,-13.37,35.71
MISC,5.16,-18.99,88.61
HEALTHCARE,5.11,3.54,20.82
ELECTRICAL,4.81,-10.15,57.85
AUTO,4.30,-18.34,69.83


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2767894.0,23
XR,1102179.0,13
AR,1047720.0,9
X40,822786.0,10
X40N,594663.0,12
XY25,491560.0,8
OX40N,343404.0,10
SR,185030.0,2
X5K,149661.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2692502.0,30
M-SC,1360244.0,15
H-LC,1296730.0,20
H-MC,1158789.0,15
M-LC,450238.0,5
M-MC,331966.0,2
L-SC,270121.0,3
L-MC,57301.0,1
L-LC,51187.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,927028.0,7
M-SC,XY24,835760.0,7
H-SC,AR,611351.0,4
H-LC,X40,549361.0,6
H-SC,XR,516451.0,6
H-MC,XY24,446463.0,4
M-MC,XY24,331966.0,2
H-LC,X40N,298030.0,6
H-SC,X40N,222147.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
